In [9]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

In [2]:
sentence = ("if you want to build a ship, don't drum up people together to "
            "collect wood and don't assign them tasks and work, but rather "
            "teach them to long for the endless immensity of the sea.")

### 문자 집합을 생성하고, 각 문자에 고유한 정수를 부여

In [3]:
char_set = list(set(sentence)) # 중복을 제거한 문자 집합 생성
char_dic = {c: i for i, c in enumerate(char_set)} # 각 문자에 정수 인코딩

문자 집합의 크기는 25이며, 입력을 원-핫 벡터로 사용할 것이므로 이는 매 시점마다 들어갈 입력의 크기이기도 합니다.

In [5]:
print(char_dic) # 공백도 여기서는 하나의 원소
dic_size = len(char_dic)
print('문자 집합의 크기 : {}'.format(dic_size))

{'u': 0, 't': 1, 'k': 2, 's': 3, 'f': 4, 'l': 5, 'h': 6, 'e': 7, ',': 8, 'y': 9, "'": 10, 'a': 11, 'p': 12, 'o': 13, 'c': 14, 'n': 15, 'i': 16, 'd': 17, 'b': 18, '.': 19, 'm': 20, ' ': 21, 'r': 22, 'w': 23, 'g': 24}
문자 집합의 크기 : 25


### 하이퍼파라미터를 설정
sequence_length라는 변수를 선언했는데, 우리가 앞서 만든 샘플을 10개 단위로 끊어서 샘플을 만들 예정이기 때문입니다.

In [6]:
# 하이퍼파라미터 설정
hidden_size = dic_size # hidden size는 마음대로 변경 가능
sequence_length = 10  # 임의 숫자 지정
learning_rate = 0.1

임의로 지정한 sequence_length 값인 10의 단위로 샘플들을 잘라서 데이터를 만드는 모습을 보여줍니다.

In [7]:
# 데이터 구성
x_data = []
y_data = []

for i in range(0, len(sentence) - sequence_length):
    x_str = sentence[i:i + sequence_length]
    y_str = sentence[i + 1: i + sequence_length + 1] # 한 칸씩 쉬프트 된 시퀀스
    print(i, x_str, '->', y_str)

    x_data.append([char_dic[c] for c in x_str])  # x str to index
    y_data.append([char_dic[c] for c in y_str])  # y str to index

0 if you wan -> f you want
1 f you want ->  you want 
2  you want  -> you want t
3 you want t -> ou want to
4 ou want to -> u want to 
5 u want to  ->  want to b
6  want to b -> want to bu
7 want to bu -> ant to bui
8 ant to bui -> nt to buil
9 nt to buil -> t to build
10 t to build ->  to build 
11  to build  -> to build a
12 to build a -> o build a 
13 o build a  ->  build a s
14  build a s -> build a sh
15 build a sh -> uild a shi
16 uild a shi -> ild a ship
17 ild a ship -> ld a ship,
18 ld a ship, -> d a ship, 
19 d a ship,  ->  a ship, d
20  a ship, d -> a ship, do
21 a ship, do ->  ship, don
22  ship, don -> ship, don'
23 ship, don' -> hip, don't
24 hip, don't -> ip, don't 
25 ip, don't  -> p, don't d
26 p, don't d -> , don't dr
27 , don't dr ->  don't dru
28  don't dru -> don't drum
29 don't drum -> on't drum 
30 on't drum  -> n't drum u
31 n't drum u -> 't drum up
32 't drum up -> t drum up 
33 t drum up  ->  drum up p
34  drum up p -> drum up pe
35 drum up pe -> rum up peo
36

### 입력 데이터와 레이블 데이터를 텐서로 변환

In [11]:
x_one_hot = [np.eye(dic_size)[x] for x in x_data] # x 데이터는 원-핫 인코딩
X = torch.FloatTensor(x_one_hot)
Y = torch.LongTensor(y_data)
print('훈련 데이터의 크기 : {}'.format(X.shape))
print('레이블의 크기 : {}'.format(Y.shape))

훈련 데이터의 크기 : torch.Size([170, 10, 25])
레이블의 크기 : torch.Size([170, 10])


In [12]:
print(X[0])

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,

In [13]:
print(Y[0])

tensor([ 4, 21,  9, 13,  0, 21, 23, 11, 15,  1])


## 모델 구현

In [14]:
class Net(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, layers): # 현재 hidden_size는 dic_size와 같음.
        super(Net, self).__init__()
        self.rnn = torch.nn.RNN(input_dim, hidden_dim, num_layers=layers, batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim, hidden_dim, bias=True)

    def forward(self, x):
        x, _status = self.rnn(x)
        x = self.fc(x)
        return x

In [18]:
net = Net(dic_size, hidden_size, 2) # 이번에는 층을 두 개 쌓습니다.

옵티마이저와 손실 함수 정의

In [19]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), learning_rate)

(170, 10, 25)의 크기를 가지는데 각각 배치 차원, 시점(timesteps), 출력의 크기입니다. 나중에 정확도를 측정할 때는 이를 모두 펼쳐서 계산하게 되는데, 이때는 view를 사용하여 배치 차원과 시점 차원을 하나로 만듭니다.

In [21]:
outputs = net(X)
print(outputs.shape) # 3차원 텐서
print(outputs.view(-1, dic_size).shape) # 2차원 텐서로 변환.

torch.Size([170, 10, 25])
torch.Size([1700, 25])


레이블 데이터는 (170, 10)의 크기를 가지는데, 마찬가지로 나중에 정확도를 측정할 때는 이걸 펼쳐서 계산할 예정입니다. 이 경우 (1700)의 크기를 가지게 됩니다. 

In [22]:
print(Y.shape)
print(Y.view(-1).shape)

torch.Size([170, 10])
torch.Size([1700])


In [27]:
for i in range(10000):
    optimizer.zero_grad()
    outputs = net(X) # (170, 10, 25) 크기를 가진 텐서를 매 에포크마다 모델의 입력으로 사용
    loss = criterion(outputs.view(-1, dic_size), Y.view(-1))
    loss.backward()
    optimizer.step()

    # results의 텐서 크기는 (170, 10)
    results = outputs.argmax(dim=2)
    predict_str = ""
    for j, result in enumerate(results):
        if j == 0: # 처음에는 예측 결과를 전부 가져오지만
            predict_str += ''.join([char_set[t] for t in result])
        else: # 그 다음에는 마지막 글자만 반복 추가
            predict_str += char_set[result[-1]]
    #if i%100 == 0:
    print(predict_str)

lo                                                                                                                                                                                 
lo                                                                                                                                                                                 
lo                                                                                                                                                                                 
lo                                                                                                                                                                                 
lo                                                                                                                                                                                 
lo                                                                                                  

lo                                                                                                                                                                                 
lo                                                                                                                                                                                 
lo                                                                                                                                                                                 
lo                                                                                                                                                                                 
lo                                                                                                                                                                                 
lo                                                                                                  

lo                                                                                                                                                                                 
lo                                                                                                                                                                                 
lo                                                                                                                                                                                 
lo                                                                                                                                                                                 
lo                                                                                                                                                                                 
lo                                                                                                  

lo                                                                                                                                                                                 
lo                                                                                                                                                                                 
lo                                                                                                                                                                                 
lo                                                                                                                                                                                 
lo                                                                                                                                                                                 
lo                                                                                                  

lo                                                                                                                                                                                 
lo                                                                                                                                                                                 
lo                                                                                                                                                                                 
lo                                                                                                                                                                                 
lo                                                                                                                                                                                 
lo                                                                                                  

...

l                                                                                                                                                                                  
l                                                                                                                                                                                  
f                                                                                                                                                                                  
l                                                                                                                                                                                  
f                                                                                                                                                                                  
l                                                                                                   

l                                                                                                                                                                                  
l                                                                                                                                                                                  
l                                                                                                                                                                                  
l                                                                                                                                                                                  
l                                                                                                                                                                                  
l                                                                                                   

l                                                                                                                                                                                  
l                                                                                                                                                                                  
l                                                                                                                                                                                  
l                                                                                                                                                                                  
l                                                                                                                                                                                  
l                                                                                                   

l                                                                                                                                                                                  
f                                                                                                                                                                                  
l                                                                                                                                                                                  
l                                                                                                                                                                                  
l                                                                                                                                                                                  
l                                                                                                   

l                                                                                                                                                                                  
l                                                                                                                                                                                  
f                                                                                                                                                                                  
l                                                                                                                                                                                  
f                                                                                                                                                                                  
l                                                                                                   